In [1]:
import mediapipe as mp
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [2]:
mp_drawings = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [61]:
def HandTracker():
    """
    Function that returns hand tracker object
    """
    mp_drawings = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands
    hand_tracker = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.8, min_tracking_confidence=0.5)
    return hand_tracker

In [62]:
hand_tracker = HandTracker()

In [63]:
def drawLandMarks(results, image):
    """
    Function that draws land marks
    """
    mp_drawings = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands
    if results.multi_hand_landmarks:
        for num, hand in enumerate(results.multi_hand_landmarks):
            mp_drawings.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,
            # Joints Color
            mp_drawings.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
            # Line Color
            mp_drawings.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2))
    return image

In [60]:
stream = cv2.VideoCapture(0)

while stream.isOpened():
    ret, frame = stream.read()
    
    image = cv2.flip(frame, 1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hand_tracker.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = drawLandMarks(results, image)
    cv2.imshow("Data Collection", image)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

stream.release()
cv2.destroyAllWindows()

In [64]:
def extract_keypoints(results):
    '''
    Extract Hand Keypoints
    '''
    hd_keypoints = np.array([[res.x, res.y, res.z] for res in results.multi_hand_landmarks[0].landmark]).flatten() if results.multi_hand_landmarks else np.zeros(21*3)
    return hd_keypoints

In [8]:
# result_test = extract_keypoints(results)

In [17]:
!rm -rf "Hand Numbers"
!mkdir "Hand Numbers"

In [18]:
import os

In [95]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join(os.getcwd() ,'Hand Numbers') 

# Actions that we try to detect
# actions = np.array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'dot', 'plus', 'minus', 'divide', 'multiply', 'equal'])
# actions = np.array(['equal'])

# Thirty videos worth of data
no_sequences = 500

# Videos are going to be 30 frames in length
sequence_length = 5

In [96]:
actions

array(['equal'], dtype='<U5')

In [92]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [ ]:
stream = cv2.VideoCapture(0)

for action in actions:
    for sequence in range(no_sequences):
        for frame_num in range(sequence_length):
            ret, frame = stream.read()
    
            image = cv2.flip(frame, 1)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = hand_tracker.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            image = drawLandMarks(results, image)
            if frame_num == 0:
                cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('Data Collection', image)
            else:
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('Data Collection', image)
                
            # NEW Export keypoints
            keypoints = extract_keypoints(results)
            npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
            np.save(npy_path, keypoints)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    stream.release()
    cv2.destroyAllWindows()